In [1]:
using ReinforcementLearning, Intervals, Flux

const RLBase = ReinforcementLearningBase

#Need to make it so I use include("some.jl") instead but for now
#using NBInclude
#@nbinclude("RIS_Solvers.ipynb")

println("Done RL Import")

Base.@kwdef mutable struct MyGrid <: AbstractEnv
    N::Int
    #We start with squares start spread across the line at co-ordinates 1,2,3,4,5,6,7
    obs::Vector{Float32}
    max_steps::Int    
    
    rewards::Float32
    done::Bool 
    t::Int
end

function MyGrid(; n)
    println("here?")
    img = zeros(Float32, n,n)
    vect = ones(Float32, 3)
    obs = vcat(vect,vec(img))
    println(obs)
    
	MyGrid(n, obs, 10, 0, false, 0)
end

RLBase.action_space(env::MyGrid) = Base.OneTo(25)

function (env::MyGrid)(action) #This function takes the action, makes the environment step, and gives reward
    
    env.obs[action[1]] = 6
    
    env.rewards+=1
    
    env.t+=1
    #Max steps check
    if env.t >= env.max_steps
        env.done = true
    end
end

RLBase.state(env::MyGrid, ::Observation{Vector{Float32}}) = env.obs
RLBase.state_space(env::MyGrid, ::Observation{Vector{Float32}}) = Space(fill( Interval{Int64}(0, (10*env.N) ) , (env.N^2+3)))

RLBase.is_terminated(env::MyGrid) = env.done

function RLBase.reset!(env::MyGrid)
    #Reset Counter and rewards (and termination variable)
    env.t = 0
    env.rewards = 0
    env.done = false    
    
    img = zeros(env.N,env.N)
    vect = ones(3)
    env.obs = vcat(vect,vec(img))
end;

RLBase.reward(env::MyGrid) = env.rewards

RLBase.NumAgentStyle(::MyGrid) = SINGLE_AGENT
RLBase.DynamicStyle(::MyGrid) = SEQUENTIAL
RLBase.ActionStyle(::MyGrid) = MINIMAL_ACTION_SET
RLBase.InformationStyle(::MyGrid) = PERFECT_INFORMATION

RLBase.StateStyle(::MyGrid) = Observation{Vector{Float32}}()

RLBase.RewardStyle(::MyGrid) = STEP_REWARD
RLBase.UtilityStyle(::MyGrid) = GENERAL_SUM
RLBase.ChanceStyle(::MyGrid) = DETERMINISTIC

println("Are we getting this far?")
N=5
env = MyGrid(; n=N)

Done RL Import
Are we getting this far?
here?
Float32[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# MyGrid

## Traits

| Trait Type        |                          Value |
|:----------------- | ------------------------------:|
| NumAgentStyle     |                  SingleAgent() |
| DynamicStyle      |                   Sequential() |
| InformationStyle  |           PerfectInformation() |
| ChanceStyle       |                Deterministic() |
| RewardStyle       |                   StepReward() |
| UtilityStyle      |                   GeneralSum() |
| ActionStyle       |             MinimalActionSet() |
| StateStyle        | Observation{Vector{Float32}}() |
| DefaultStateStyle | Observation{Vector{Float32}}() |

## Is Environment Terminated?

No

## State Space

`Space{Vector{Interval{Int64, Closed, Closed}}}(Interval{Int64, Closed, Closed}[Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50)])`

## Action Space

`Base.OneTo(25)`

## Current State

```
Float32[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
```


In [2]:
println("asdfghjkl;")

asdfghjkl;


In [3]:
length(state_space(env)) == length(env.obs)

true

In [4]:
UPDATE_FREQ = 32
N_ENV = 1

agent = Agent(
        policy = PPOPolicy(
            approximator = ActorCritic(
                    actor = Chain(
                                x -> ( reshape(x[4:28], (5,5,1,1)), x[1:3]),
                                Parallel(vcat, 
                                        Chain(Conv((3,3), 1 => 1, relu; stride = 1, pad = 0),
                                        Flux.flatten),

                                        Chain(Dense(3, 6, relu), Dense(6, 3, relu), Dense(3, 1, relu)) ),
                                Dense(10, 20, relu),
                                Dense(20,5,relu),
                                Dense(5,3)),

                    critic = Chain(
                                x -> ( reshape(x[4:28], (5,5,1,1)), x[1:3]),
                                Parallel(vcat, 
                                        Chain(Conv((3,3), 1 => 1, relu; stride = 1, pad = 0),
                                        Flux.flatten),

                                        Chain(Dense(3, 6, relu), Dense(6, 3, relu), Dense(3, 1, relu)) ),
                                Dense(10, 20, relu),
                                Dense(20,5,relu),
                                Dense(5,1)),
            
                optimizer = ADAM(1e-3),
            ),
            γ = 0.99f0,
            λ = 0.95f0,
            clip_range = 0.1f0,
            max_grad_norm = 0.5f0,
            n_epochs = 4,
            n_microbatches = 4,
            actor_loss_weight = 1.0f0,
            critic_loss_weight = 0.5f0,
            entropy_loss_weight = 0.001f0,
            update_freq = UPDATE_FREQ,
        ),
        trajectory = PPOTrajectory(;
            capacity = UPDATE_FREQ, 
            state = Matrix{Float32} => (28, N_ENV),
            action = Vector{Int} => (N_ENV,),
            action_log_prob = Vector{Float32} => (N_ENV,),
            reward = Vector{Float32} => (N_ENV,),  
            terminal = Vector{Bool} => (N_ENV,),
        ),
    )


#stop_condition = StopAfterStep(30000, is_show_progress=true)
hook = TotalBatchRewardPerEpisode(N_ENV)

TotalBatchRewardPerEpisode([Float64[]], [0.0], true)

In [5]:
run(agent, env, StopAfterEpisode(8), hook)

Progress:  25%|██████████▎                              |  ETA: 0:00:51

LoadError: BoundsError: attempt to access 1-element view(::Matrix{Float32}, 1, :) with eltype Float32 at index [33]